In [19]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import sqlite3
from sqlite3 import Error


rcParams.update({'figure.autolayout': True})

# Create a database connection
conn = None 
try:
    conn = sqlite3.connect('../spotify.db')
    print("Connected to the database")
except Error as error:
    print(error)


Connected to the database


In [20]:
# Query top 10 songs by artist in terms of duration_ms
  
cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS top_10_songs_by_duration_per_artist;")
create_view = """
    CREATE VIEW IF NOT EXISTS top_10_songs_by_duration_per_artist
    AS
    SELECT artist_name, song_name, duration_ms, minutes
    FROM (
        SELECT
            a.artist_name,
            t.song_name,
            t.duration_ms,
            strftime('%H:%M:%S', t.duration_ms/1000, 'unixepoch') minutes,
            row_number() over (
                PARTITION BY a.artist_name 
                ORDER BY a.artist_name, t.duration_ms DESC
            ) as longest_duration
        FROM track t
        JOIN album alb
            ON alb.album_id = t.album_id
        JOIN artist a
            ON a.artist_id = alb.artist_id) duration_ranks
    WHERE longest_duration <= 10;
"""
cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM top_10_songs_by_duration_per_artist;
"""

pd.read_sql_query(select_query, conn)

,artist_name,song_name,duration_ms,minutes
0,Bad Bunny,Andrea,339611,00:05:39
1,Bad Bunny,La Romana,300579,00:05:00
2,Bad Bunny,Safaera,295176,00:04:55
3,Bad Bunny,RLNDT,284852,00:04:44
4,Bad Bunny,Ojitos Lindos,258298,00:04:18
...,...,...,...,...
195,Van Halen,You Really Got Me / Cabo Wabo - Live,478360,00:07:58
196,Van Halen,Once,462733,00:07:42
197,Van Halen,Cabo Wabo,422266,00:07:02
198,Van Halen,Pleasure Dome,416040,00:06:56


In [21]:
# Query top 20 artists in the db by the number of followers
 
cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS top_20_artists_by_followers;")
create_view = """
    CREATE VIEW IF NOT EXISTS top_20_artists_by_followers
    AS
    SELECT
        a.artist_name,
        a.followers,
        a.popularity,
        a.genre,
        COUNT(*) num_albums,
        STRFTIME('%m-%d-%Y', MAX(alb.release_date)) latest_album_release_date
    FROM artist a
    JOIN album alb
        ON alb.artist_id = a.artist_id
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 20;   
"""
cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM top_20_artists_by_followers;
"""

pd.read_sql_query(select_query, conn)

,artist_name,followers,popularity,genre,num_albums,latest_album_release_date
0,Billie Eilish,68972660,89,art pop,2,07-30-2021
1,Drake,67153280,95,canadian hip hop,12,06-17-2022
2,Bad Bunny,56281874,100,reggaeton,5,05-06-2022
3,Rihanna,50150393,88,barbadian pop,13,01-28-2016
4,Queen,42806099,84,classic rock,24,10-19-2018
5,J Balvin,35328543,88,reggaeton,9,09-10-2021
6,KAROL G,28676636,87,reggaeton,3,03-26-2021
7,J. Cole,17923010,85,conscious hip hop,7,05-14-2021
8,Pink Floyd,17537356,78,album rock,23,09-16-2022
9,Nirvana,15976111,81,alternative rock,12,11-01-2019


In [22]:
# Query top 10 songs by artists in terms of tempo
  
cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS top_10_songs_by_tempo_per_artist;")
create_view = """
    CREATE VIEW IF NOT EXISTS top_10_songs_by_tempo_per_artist
    AS
    SELECT artist_name, song_name, tempo, album_name
    FROM (
        SELECT
            a.artist_name,
            t.song_name,
            printf('%.3f', f.tempo) tempo,
            alb.album_name,
            row_number() over (
                PARTITION BY a.artist_name
                ORDER BY a.artist_name, f.tempo DESC
            ) as highest_tempo
        FROM track_feature f
        JOIN track t
            ON t.track_id = f.track_id
        JOIN album alb
            ON alb.album_id = t.album_id
        JOIN artist a
            ON a.artist_id = alb.artist_id) tempo_ranks
    WHERE highest_tempo <= 10;
"""
cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM top_10_songs_by_tempo_per_artist;
"""

pd.read_sql_query(select_query, conn)

# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,artist_name,song_name,tempo,album_name
0,Bad Bunny,La Corriente,196.120,Un Verano Sin Ti
1,Bad Bunny,SORRY PAPI,191.911,EL ÚLTIMO TOUR DEL MUNDO
2,Bad Bunny,Un Verano Sin Ti,187.844,Un Verano Sin Ti
3,Bad Bunny,Solo de Mi,179.972,X 100PRE
4,Bad Bunny,La Difícil,179.870,YHLQMDLG
...,...,...,...,...
195,Van Halen,A.F.U. (Naturally Wired),195.974,OU812
196,Van Halen,Outta Space,186.127,A Different Kind Of Truth
197,Van Halen,Year to the Day,185.406,Van Halen III
198,Van Halen,Top Jimmy - 2015 Remaster,184.484,1984 (Remastered)


In [23]:
# Query how many songs and albums each artist has 
  
cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS num_songs_albums_by_artist;")
create_view = """
    CREATE VIEW IF NOT EXISTS num_songs_albums_by_artist
    AS
    SELECT
        a.artist_name,
        COUNT(alb.album_name) total_albums,
        SUM(alb.total_tracks) total_songs
    FROM album alb
    JOIN artist a
        ON a.artist_id = alb.artist_id 
    GROUP BY 1
    ORDER BY 2 DESC, 3 DESC, 1;
"""
cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM num_songs_albums_by_artist;
"""

pd.read_sql_query(select_query, conn)

,artist_name,total_albums,total_songs
0,Lynyrd Skynyrd,34,538
1,Queen,24,408
2,Pink Floyd,23,314
3,Black Sabbath,22,220
4,Led Zeppelin,16,205
5,Van Halen,14,176
6,Rihanna,13,164
7,Foo Fighters,13,159
8,Nirvana,12,220
9,Drake,12,206


In [24]:
# Query the albums that were released in the 1990's
   
cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS albums_released_in_90s;")
create_view = """
    CREATE VIEW IF NOT EXISTS albums_released_in_90s
    AS
    SELECT
        strftime('%m-%d-%Y', alb.release_date) release_date,
        alb.album_name,
        a.artist_name
    FROM album alb
    JOIN artist a
        ON a.artist_id = alb.artist_id 
    WHERE strftime('%Y', alb.release_date) BETWEEN '1990' AND '2000'
    ORDER BY alb.release_date, 2, 3;
"""
cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM albums_released_in_90s;
"""

pd.read_sql_query(select_query, conn)

,release_date,album_name,artist_name
0,02-04-1991,Innuendo,Queen
1,06-17-1991,For Unlawful Carnal Knowledge,Van Halen
2,09-26-1991,Nevermind,Nirvana
3,09-26-1991,Nevermind (Remastered),Nirvana
4,11-12-1991,Lynyrd Skynyrd,Lynyrd Skynyrd
5,01-01-1992,Incesticide,Nirvana
6,05-26-1992,Live At Wembley Stadium,Queen
7,06-22-1992,Dehumanizer (2008 Remaster),Black Sabbath
8,02-02-1993,The Last Rebel,Lynyrd Skynyrd
9,02-23-1993,"Van Halen Live: Right Here, Right Now",Van Halen


In [25]:
# Query top 20 songs by danceability 

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS top_20_songs_by_danceability;")
create_view = """
    CREATE VIEW IF NOT EXISTS top_20_songs_by_danceability
    AS
    SELECT
        a.artist_name,
        t.song_name,
        printf('%.3f', f.danceability) danceability
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f
        ON f.track_id = t.track_id
    WHERE f.danceability >= 0.80
    ORDER BY 3 DESC
    LIMIT 20;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM top_20_songs_by_danceability;
"""

pd.read_sql_query(select_query, conn)

# A lot of Jack Harlow! 
# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,artist_name,song_name,danceability
0,Jack Harlow,I Got A Shot,0.973
1,Becky G,NO MIENTEN,0.959
2,Jack Harlow,WALK IN THE PARK,0.956
3,Jack Harlow,SMELLS LIKE INCENSE,0.952
4,Jack Harlow,2STYLISH,0.945
5,Jack Harlow,Way Out (feat. Big Sean),0.945
6,Jack Harlow,Ice Cream,0.943
7,Natti Natasha,Qué Lío,0.939
8,Billie Eilish,my strange addiction,0.939
9,Becky G,FULANITO,0.938


In [26]:
# Query average energy level of each artist 

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS avg_energy_of_artist;")
create_view = """
    CREATE VIEW IF NOT EXISTS avg_energy_of_artist
    AS
    SELECT
        a.artist_name,
        a.genre,
        printf('%.3f', AVG(f.energy)) energy
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f
        ON f.track_id = t.track_id
    GROUP BY 1
    ORDER BY 3 DESC;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM avg_energy_of_artist;
"""

pd.read_sql_query(select_query, conn)

# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,artist_name,genre,energy
0,Nirvana,alternative rock,0.814
1,Van Halen,album rock,0.812
2,Natti Natasha,latin hip hop,0.768
3,Britney Spears,dance pop,0.744
4,Foo Fighters,alternative metal,0.728
5,J Balvin,reggaeton,0.716
6,Lynyrd Skynyrd,album rock,0.713
7,KAROL G,reggaeton,0.713
8,Black Sabbath,album rock,0.706
9,Led Zeppelin,album rock,0.699


In [27]:
# Query artists with at least 20 albums

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS artists_w_atleast_20_albums;")
create_view = """
    CREATE VIEW IF NOT EXISTS artists_w_atleast_20_albums
    AS
    SELECT
        a.artist_name,
        COUNT(alb.album_name) total_albums
    FROM album alb
    JOIN artist a
        ON a.artist_id = alb.artist_id 
    GROUP BY 1
    HAVING COUNT(alb.album_name) > 20
    ORDER BY 2 DESC, 1;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM artists_w_atleast_20_albums;
"""

pd.read_sql_query(select_query, conn)

,artist_name,total_albums
0,Lynyrd Skynyrd,34
1,Queen,24
2,Pink Floyd,23
3,Black Sabbath,22


In [28]:
# Query average audio features by genre

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS avg_audio_features_by_genre;")
create_view = """
    CREATE VIEW IF NOT EXISTS avg_audio_features_by_genre
    AS
    SELECT
        a.genre,
        printf('%.1f', AVG(a.popularity)) popularity,
        printf('%.3f', AVG(f.danceability)) danceability,
        printf('%.3f', AVG(f.energy)) energy,
        printf('%.3f', AVG(f.instrumentalness)) instrumentalness, 
        printf('%.3f', AVG(f.liveness)) liveness,
        printf('%.3f', AVG(f.loudness)) loudness,
        printf('%.3f', AVG(f.speechiness)) speechiness,
        printf('%.3f', AVG(f.tempo)) tempo,
        printf('%.3f', AVG(f.valence)) valence 
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id 
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f 
        ON f.track_id = t.track_id
    GROUP BY 1
    ORDER BY 2 DESC;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM avg_audio_features_by_genre;
"""

pd.read_sql_query(select_query, conn)

# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,genre,popularity,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,canadian hip hop,95.0,0.660,0.556,0.016,0.190,-8.009,0.216,119.418,0.362
1,reggaeton,91.7,0.741,0.696,0.009,0.174,-5.706,0.121,126.559,0.617
2,art pop,89.0,0.604,0.299,0.186,0.190,-13.389,0.143,107.392,0.297
3,barbadian pop,88.0,0.625,0.694,0.019,0.186,-5.737,0.089,121.571,0.521
4,conscious hip hop,85.0,0.634,0.603,0.003,0.287,-8.535,0.273,105.473,0.368
5,classic rock,84.0,0.389,0.688,0.087,0.453,-9.490,0.096,118.834,0.346
6,deep underground hip hop,83.0,0.791,0.577,0.004,0.150,-7.321,0.193,117.666,0.518
7,musica mexicana,81.0,0.566,0.655,0.000,0.255,-6.350,0.105,127.867,0.665
8,alternative rock,81.0,0.399,0.814,0.178,0.392,-7.567,0.076,130.598,0.367
9,dance pop,80.6,0.756,0.723,0.004,0.176,-4.993,0.108,119.166,0.672


In [29]:
# Query average audio features by genre

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS loudness_energy_by_genre;")
create_view = """
    CREATE VIEW IF NOT EXISTS loudness_energy_by_genre
    AS
    SELECT
        a.genre,
        printf('%.3f', f.energy) energy,
        printf('%.3f', f.loudness) loudness
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id 
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f 
        ON f.track_id = t.track_id
    ORDER BY 1;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM loudness_energy_by_genre;
"""

pd.read_sql_query(select_query, conn)

# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,genre,energy,loudness
0,album rock,0.341,-14.551
1,album rock,0.632,-12.507
2,album rock,0.224,-14.926
3,album rock,0.506,-11.746
4,album rock,0.324,-16.001
...,...,...,...
3299,reggaeton,0.725,-5.497
3300,reggaeton,0.379,-10.348
3301,reggaeton,0.528,-6.554
3302,reggaeton,0.536,-6.663


In [30]:
# Query average audio features by artist

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS avg_audio_features_by_artist;")
create_view = """
    CREATE VIEW IF NOT EXISTS avg_audio_features_by_artist
    AS
    SELECT
        a.artist_name Artist,
        printf('%.3f', AVG(f.danceability)) Danceability,
        printf('%.3f', AVG(f.energy)) Energy,
        printf('%.3f', AVG(f.tempo)) Tempo,
        printf('%.3f', AVG(f.speechiness)) Speechiness,
        printf('%.3f', AVG(f.instrumentalness)) Instrumentalness, 
        printf('%.3f', AVG(f.valence)) Valence 
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id 
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f 
        ON f.track_id = t.track_id
    GROUP BY a.artist_id
    ORDER BY 1;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM avg_audio_features_by_artist;
"""

pd.read_sql_query(select_query, conn)

# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,Artist,Danceability,Energy,Tempo,Speechiness,Instrumentalness,Valence
0,Bad Bunny,0.751,0.653,120.205,0.115,0.000,0.488
1,Becky G,0.786,0.695,122.707,0.157,0.006,0.623
2,Billie Eilish,0.604,0.299,107.392,0.143,0.186,0.297
3,Black Sabbath,0.332,0.706,121.610,0.058,0.164,0.363
4,Britney Spears,0.733,0.744,116.509,0.070,0.003,0.708
5,Drake,0.660,0.556,119.418,0.216,0.016,0.362
6,Foo Fighters,0.458,0.728,130.410,0.095,0.079,0.423
7,Grupo Firme,0.566,0.655,127.867,0.105,0.000,0.665
8,J Balvin,0.732,0.716,130.893,0.127,0.018,0.696
9,J. Cole,0.634,0.603,105.473,0.273,0.003,0.368


In [31]:
# Query average audio features by artist

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS audio_features_correlations;")
create_view = """
    CREATE VIEW IF NOT EXISTS audio_features_correlations
    AS
    SELECT
        printf('%.3f', f.danceability) Danceability,
        printf('%.3f', f.energy) Energy,
        printf('%.3f', f.tempo) Tempo,
        printf('%.3f', f.speechiness) Speechiness,
        printf('%.3f', f.instrumentalness) Instrumentalness, 
        printf('%.3f', f.valence) Valence 
    FROM track_feature f
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM audio_features_correlations;
"""

pd.read_sql_query(select_query, conn)

# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,Danceability,Energy,Tempo,Speechiness,Instrumentalness,Valence
0,0.558,0.671,80.436,0.292,0.000,0.485
1,0.193,0.804,142.820,0.091,0.644,0.053
2,0.619,0.738,139.639,0.218,0.000,0.354
3,0.241,0.188,71.478,0.040,0.890,0.083
4,0.256,0.966,134.451,0.074,0.001,0.325
...,...,...,...,...,...,...
3379,0.886,0.646,98.030,0.189,0.000,0.867
3380,0.316,0.797,175.037,0.069,0.161,0.539
3381,0.710,0.304,98.037,0.033,0.000,0.312
3382,0.792,0.421,89.994,0.070,0.004,0.542


In [32]:
# Query audio features for Jack Harlow's album, Come Home The Kids Miss You
# I was curious about this since his album had a lot of songs with high danceability 

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS audio_features_for_album;")
create_view = """
    CREATE VIEW IF NOT EXISTS audio_features_for_album
    AS
    SELECT
        t.song_name,
        printf('%.3f', f.danceability) danceability,
        printf('%.3f', f.energy) energy,
        printf('%.3f', f.instrumentalness) instrumentalness, 
        printf('%.3f', f.liveness) liveness,
        printf('%.3f', f.loudness) loudness,
        printf('%.3f', f.speechiness) speechiness,
        printf('%.3f', f.tempo) tempo,
        printf('%.3f', f.valence) valence 
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id 
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f 
        ON f.track_id = t.track_id
    WHERE alb.album_id = '2eE8BVirX9VF8Di9hD90iw'
    ORDER BY 1;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM audio_features_for_album;
"""

pd.read_sql_query(select_query, conn)

# printf to make it more easily readable 
# but it changes the datatype so we will have to convert it back in our visualizations  

,song_name,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,Churchill Downs (feat. Drake),0.710,0.522,0.000,0.110,-9.102,0.360,96.448,0.370
1,Dua Lipa,0.833,0.652,0.099,0.106,-6.942,0.076,158.022,0.405
2,First Class,0.902,0.582,0.000,0.111,-5.902,0.109,107.005,0.332
3,I Got A Shot,0.973,0.433,0.000,0.075,-6.605,0.089,120.055,0.933
4,I'd Do Anything To Make You Smile,0.832,0.522,0.079,0.377,-7.244,0.050,150.044,0.253
5,Like A Blade Of Grass,0.891,0.489,0.000,0.090,-8.140,0.135,93.515,0.794
6,Lil Secret,0.779,0.653,0.000,0.096,-5.977,0.206,69.056,0.841
7,Movie Star (feat. Pharrell Williams),0.829,0.488,0.000,0.107,-7.275,0.183,93.199,0.551
8,Nail Tech,0.677,0.557,0.000,0.098,-8.409,0.069,150.127,0.100
9,Parent Trap (feat. Justin Timberlake),0.878,0.523,0.000,0.116,-7.653,0.120,92.050,0.422


In [33]:
# Query popularity by artists

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS popularity_by_artists_in_genre;")
create_view = """
    CREATE VIEW IF NOT EXISTS popularity_by_artists_in_genre
    AS
    SELECT
        a.artist_name,
        a.genre,
        a.popularity
    FROM artist a
    ORDER BY 3 DESC;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM popularity_by_artists_in_genre;
"""

pd.read_sql_query(select_query, conn)

,artist_name,genre,popularity
0,Bad Bunny,reggaeton,100
1,Drake,canadian hip hop,95
2,Billie Eilish,art pop,89
3,J Balvin,reggaeton,88
4,Rihanna,barbadian pop,88
5,KAROL G,reggaeton,87
6,J. Cole,conscious hip hop,85
7,Queen,classic rock,84
8,Jack Harlow,deep underground hip hop,83
9,Grupo Firme,musica mexicana,81


In [34]:
# Query tempos of songs by genre 

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS tempos_by_genre;")
create_view = """
    CREATE VIEW IF NOT EXISTS tempos_by_genre
    AS
    SELECT
        a.genre,
        CAST(f.tempo AS INT) tempo,
        a.artist_name,
        t.song_name
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id 
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f 
        ON f.track_id = t.track_id
    WHERE f.tempo != 0
    ORDER BY 1, 2 DESC, 3;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM tempos_by_genre;
"""

pd.read_sql_query(select_query, conn)

,genre,tempo,artist_name,song_name
0,album rock,213,Van Halen,The Full Bug - 2015 Remaster
1,album rock,208,Lynyrd Skynyrd,I Know A Little - Live
2,album rock,208,Pink Floyd,Nervana
3,album rock,206,Black Sabbath,Over to You - 2014 Remaster
4,album rock,206,Pink Floyd,Nobody Home
...,...,...,...,...
3294,reggaeton,79,Bad Bunny,CANCIÓN CON YANDEL
3295,reggaeton,79,J Balvin,UN PESO
3296,reggaeton,78,Bad Bunny,Después de la Playa
3297,reggaeton,76,Bad Bunny,EL MUNDO ES MÍO


In [35]:
# Query valence vs popularity by genre

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS valence_popularity_by_genre;")
create_view = """
    CREATE VIEW IF NOT EXISTS valence_popularity_by_genre
    AS
    SELECT
        a.genre,
        printf('%.3f', AVG(f.valence)) valence,
        printf('%.1f', AVG(a.popularity)) popularity
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id 
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f 
        ON f.track_id = t.track_id
    GROUP BY 1
    ORDER BY 1;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM valence_popularity_by_genre;
"""

pd.read_sql_query(select_query, conn)

,genre,valence,popularity
0,album rock,0.451,74.0
1,alternative metal,0.423,78.0
2,alternative rock,0.367,81.0
3,art pop,0.297,89.0
4,barbadian pop,0.521,88.0
5,canadian hip hop,0.362,95.0
6,classic rock,0.346,84.0
7,conscious hip hop,0.368,85.0
8,dance pop,0.672,80.6
9,deep underground hip hop,0.518,83.0


In [36]:
# Query valence vs popularity by artist

cur = conn.cursor()
cur.execute("DROP VIEW IF EXISTS valence_popularity_by_artist;")
create_view = """
    CREATE VIEW IF NOT EXISTS valence_popularity_by_artist
    AS
    SELECT
        a.artist_name artist,
        printf('%.3f', AVG(f.valence)) valence,
        printf('%.1f', AVG(a.popularity)) popularity
    FROM artist a
    JOIN album alb
        ON a.artist_id = alb.artist_id 
    JOIN track t
        ON t.album_id = alb.album_id
    JOIN track_feature f 
        ON f.track_id = t.track_id
    GROUP BY 1
    ORDER BY 1;
"""

cur.execute(create_view)
conn.commit()

select_query = """
    SELECT *
    FROM valence_popularity_by_artist;
"""

pd.read_sql_query(select_query, conn)

,artist,valence,popularity
0,Bad Bunny,0.488,100.0
1,Becky G,0.623,80.0
2,Billie Eilish,0.297,89.0
3,Black Sabbath,0.363,72.0
4,Britney Spears,0.708,81.0
5,Drake,0.362,95.0
6,Foo Fighters,0.423,78.0
7,Grupo Firme,0.665,81.0
8,J Balvin,0.696,88.0
9,J. Cole,0.368,85.0
